In [6]:
!pip3 install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 24.3 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
select
i.slug, -- that would be the ID output
i.name,
legalentityname,
searchaliases, -- all aliases
description, -- just normal description, most likely from marketing
structured_description, -- AI description made by the orange team ~500 issuer backfilled, might need to join with the syntetic dataset
lifecyclestatus, -- might not be 100% accurate we can do a join to find it
bannermessage,-- manually curate, it means something happened to this company and we want to tell customers
sub_sector.name as sub_sector,
sector.name as sector,
website,
crunchbaseurl,
yearfounded, -- might need to actually define recency buckets or tiers
archivedat, -- maybe don't use for wieghts, we mgiht only drop archived issuers that don't have trades or IOIs and make a new label from it,
domicilecountrycode, -- as is is not useful we might need to check how relevant companies cluster, examplke US-DE (delaware) are more likely than other combinations
domicilestatecode,
city,
extra_data.forge_price_issuer_tier -- if tier_1, then better than others
from source.icms_public.issuer as i
inner join source.icms_public.subsector as sub_sector on sub_sector.id=i.subsectorid 
inner join source.icms_public.sector as sector on sub_sector.sectorid=sector.id
inner join source.icms_public.issuer_data_distribution extra_data on extra_data.issuer_slug=i.slug ;


select fr.issuer_slug,
array_agg(funding_date), -- this can be used for recency 
array_agg(fr.share_type), -- this can be used for labeling, and tiering, if a company as at least C is more likely to be traded
from source.icms_public.funding_rounds as fr
inner join source.icms_public.issuer_waterfall_data wd on wd.id=fr.waterfall_data_id and wd.status='Published'
where fr.share_type not in ('Common', 'Common Options')
group by fr.issuer_slug;


-- last_pending_trade_date might show changes every week last_closed_trade_date will show changes every few weeks and months
select 
slug,
count(id) as num_trades,
max(closed_date) as last_closed_trade_date, -- sometimes is present if the transaciton is closed
max(transaction_accepted_date) as last_pending_trade_date, --sometimes is there for pending trades
from
fdp.refined.index_transactions
group by slug
order by 4 DESC;

select 
issuer_slug,
forge_price, -- higher the better 
forge_implied_valuation , -- higher the better, most important field for ranking 
forge_price_source_external, -- in order of importance 1: VWAP, 2: IOIs, 3: Primary, the other terms have negative impact
ioi_price is not null as has_iois
from fdp.refined.index_forge_price
where calc_date = '2024-10-10'
and forge_price is not null;

The columns:
ISSUER_SLUG	NAME	LEGALENTITYNAME	SEARCHALIASES	DESCRIPTION	STRUCTURED_DESCRIPTION	LIFECYCLESTATUS	BANNERMESSAGE	SUB_SECTOR	SECTOR	WEBSITE	CRUNCHBASEURL	YEARFOUNDED	ARCHIVEDAT	DOMICILECOUNTRYCODE	DOMICILESTATECODE	CITY	FORGE_PRICE_ISSUER_TIER	PRICE	IMPLIED_VALUATION	PRICE_SOURCE_EXTERNAL	HAS_IOIS	ARRAY_AGG(FUNDING_DATE)	ARRAY_AGG(FR.SHARE_TYPE)	NUM_TRADES	LAST_CLOSED_TRADE_DATE	LAST_PENDING_TRADE_DATE

A sample row:
floqast	FloQast	FloQast Inc.	[]	FloQast provides cloud-based software that helps to streamline accounting departments by centralizing the workflow, assigning tasks, maintaining supporting documentation and automating reconciliations. This company was founded by Cullen Zandstra and Michael Whitmire in 2013 and is headquartered in Los Angeles, CA. FloQast specializes in the fields of accounting, financial services, and software as a service.				Business Operations	Enterprise Software	floqast.com	https://www.crunchbase.com/organization/floqast	2013		US	DE		TIER_1	16.4	1584400875	VWAP	FALSE	"[
  ""2024-04-10 00:00:00.000"",
  ""2024-04-10 00:00:00.000"",
  ""2018-12-20 00:00:00.000"",
  ""2014-11-05 00:00:00.000"",
  ""2016-03-16 00:00:00.000"",
  ""2017-06-21 00:00:00.000"",
  ""2021-07-21 00:00:00.000"",
  ""2020-01-21 00:00:00.000""
]"	"[
  ""Series E"",
  ""Series E1"",
  ""Series B2"",
  ""Series A"",
  ""Series A2"",
  ""Series B"",
  ""Series D"",
  ""Series C""
]"	5	8/28/24	6/12/24

ToDo:
- tfidf for columns 
- also higher weights for below: 
  - US in domicilecountrycode, 
  - DE in domicilestatecode,   
  - "Series C” and higher in ARRAY_AGG(FR.SHARE_TYPE), -- https://support.crunchbase.com/hc/en-us/articles/115010458467-Glossary-of-Funding-Types
  - recency in ARRAY_AGG(FUNDING_DATE), implied_valuation higher the better - most important field for ranking, 
  - FORGE_PRICE_SOURCE_EXTERNAL in order of importance 1: VWAP, 2: IOIs, 3: Primary, the other terms have negative impact.
  - FORGE_PRICE_ISSUER_TIER , if it is TIER_1, higher weights
  - FORGE_PRICE, the higher the better
  - FORGE_IMPLIED_VALUATION, the higher the better

not yet decided what to do with this???
  - LIFECYCLESTATUS 
  - HAS_IOIS
  - NUM_TRADES
  - LAST_CLOSED_TRADE_DATE	
  - LAST_PENDING_TRADE_DATE


*-* the columns with text values can help with matching, such as searchaliases

* The input is a word or sentence, and the output is the issuer_slug. 

In [46]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from scipy.sparse import hstack
import re
from datetime import datetime
import ast
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut


In [33]:

df_forge_price = pd.read_csv('data/forge_price_sample.csv')
df_funding_rounds = pd.read_csv('data/funding-rounds.csv')
df_issuers = pd.read_csv('data/issuers.csv')
df_trade_facts = pd.read_csv('data/trade_facts.csv')


df_all = df_issuers.merge(df_forge_price, on='ISSUER_SLUG', how='left') \
            .merge(df_funding_rounds, on='ISSUER_SLUG', how='left') \
            .merge(df_trade_facts, on='ISSUER_SLUG', how='left')


print(df_all.head)

df_all.to_csv('data/merged_result.csv', index=False)


<bound method NDFrame.head of           ISSUER_SLUG             NAME        LEGALENTITYNAME SEARCHALIASES  \
0           tradealgo        TradeAlgo        TA Fintech Inc.            []   
1            latitude         Latitude       Oasis Tech, Inc.            []   
2           revery-ai        Revery AI         Revery AI Inc.            []   
3                soci             SOCi             SOCi, Inc.            []   
4           colossyan        Colossyan         Colossyan Inc.            []   
...               ...              ...                    ...           ...   
5291          vetster          Vetster          Vetster, Inc.            []   
5292          embrace          Embrace   Embrace Mobile, Inc.            []   
5293  particle-health  Particle Health  Particle Health, Inc.            []   
5294       leaf-trade       Leaf Trade        Leaf Trade Inc.            []   
5295        empowerly        Empowerly         Empowerly, Inc            []   

                     

In [54]:
weighted_df = pd.DataFrame()

In [42]:

def weight_domicile_country_code(code):
    return 1 if code == 'US' else 0

def weight_domicile_state_code(code):
    return 1 if code == 'DE' else 0


funding_round_ranking = {
    'seed': 0,
    'series a': 1,
    'series b': 2,
    'series c': 3,
    'series d': 4,
    'series e': 5,
    'series f': 6,
    'series g': 7,
    'series h': 8,
    'series i': 9
}
def normalize_and_weight_series(series_types):  
    if pd.isna(series_types) or not series_types:  # Handle NaN or empty lists
        return 0  # Neutral weight for missing values or empty lists

    normalized_weights = []
    
    for series in series_types:
        series_lower = series.lower().strip()
        
        match = re.search(r'series [a-z]', series_lower)
        if match:
            normalized_type = match.group()
            weight = funding_round_ranking.get(normalized_type, 0)
            normalized_weights.append(weight)
        else:
            # Fallback for unrecognized series types (neutral)
            normalized_weights.append(0)  
    
    return max(normalized_weights)  # Return the highest weight in the list


def weight_recency(funding_dates_str):
    if pd.isna(funding_dates_str) or not funding_dates_str: # Handle NaN or empty lists
        return -99999  # Lowest weight for missing or empty values
    
    funding_dates_clean = re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}', funding_dates_str)
    funding_dates = [datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f') for date in funding_dates_clean]
    most_recent_date = max(funding_dates)
    now = datetime.now()
    recency_weight = (now - most_recent_date).days
    return -recency_weight  # More recent = higher weight (less days, more weight)


def weight_price_source(price_source):
    if pd.isna(price_source):  # Handle NaN or missing values
        return -0.1  # Assign negative weight to missing values
    price_source_lower = price_source.lower().strip()
    
    if re.search(r'vwap', price_source_lower):
        return 1  
    elif re.search(r'iois', price_source_lower):
        return 0.5  
    elif re.search(r'primary', price_source_lower):
        return 0.2  
    else:
        return -0.1  # Negative weight for other or unrecognized types

def weight_price_issuer_tier(tier):
    if pd.isna(tier):  # Handle NaN or missing values
        return 0  # Neutral weight for missing
    tier_lower = tier.lower().strip()
    if tier_lower == 'tier_1':
        return 1  # Highest weight for TIER_1
    else:
        return 0.5  # Neutral or medium weight for other tiers


def weight_price(price):
    if pd.isna(price):  # Handle NaN or missing values
        return 0  # Neutral weight for missing values
    return price  # Higher value means higher weight

def weight_implied_valuation(valuation):
    if pd.isna(valuation):  # Handle NaN or missing values
        return 0  # Neutral weight for missing values
    return valuation  # Higher value means higher weight


In [55]:
# Apply the weights to the dataframe
weighted_df['DOMICILECOUNTRYCODE_WEIGHT'] = df_all['DOMICILECOUNTRYCODE'].apply(weight_domicile_country_code)
weighted_df['DOMICILESTATECODE_WEIGHT'] = df_all['DOMICILESTATECODE'].apply(weight_domicile_state_code)
weighted_df['SHARE_TYPE_WEIGHT'] = df_all['ARRAY_AGG(FR.SHARE_TYPE)'].apply(normalize_and_weight_series)
weighted_df['FUNDING_DATE_WEIGHT'] = df_all['ARRAY_AGG(FUNDING_DATE)'].apply(weight_recency)
weighted_df['FORGE_PRICE_SOURCE_WEIGHT'] = df_all['FORGE_PRICE_SOURCE_EXTERNAL'].apply(weight_price_source)
weighted_df['FORGE_PRICE_ISSUER_TIER_WEIGHT'] = df_all['FORGE_PRICE_ISSUER_TIER'].apply(weight_price_issuer_tier)
weighted_df['FORGE_PRICE_WEIGHT'] = df_all['FORGE_PRICE'].apply(weight_price)
weighted_df['FORGE_IMPLIED_VALUATION_WEIGHT'] = df_all['FORGE_IMPLIED_VALUATION'].apply(weight_implied_valuation)

# print(weighted_df.head)


In [56]:
# Convert string representation of list to list 
def clean_search_aliases(alias_str):
    if pd.isna(alias_str) or alias_str.strip() == "":
        return ""
    
    alias_list = ast.literal_eval(alias_str)
    return ' '.join(alias_list)  

df_all['SEARCHALIASES_CLEANED'] = df_all['SEARCHALIASES'].apply(clean_search_aliases)


In [110]:

weighted_df['COMBINED_TEXT'] = df_all[['NAME', 'SEARCHALIASES_CLEANED', 'DESCRIPTION', 'STRUCTURED_DESCRIPTION', 
                          'SUB_SECTOR', 'SECTOR', 'WEBSITE', 'CRUNCHBASEURL', 'BANNERMESSAGE']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [111]:
weighted_df['ISSUER_SLUG'] = df_all['ISSUER_SLUG']
weighted_df['LEGALENTITYNAME'] = df_all['LEGALENTITYNAME']

??? LIFECYCLESTATUS		YEARFOUNDED		ARCHIVEDAT	CITY	HAS_IOIS	NUM_TRADES	LAST_CLOSED_TRADE_DATE	LAST_PENDING_TRADE_DATE

In [123]:
weighted_df.to_csv('data/weighted_merged_result.csv', index=False)

In [30]:
# Handle missing values using SimpleImputer
imputer = SimpleImputer(strategy='constant', fill_value='')
df_imputed = pd.DataFrame(imputer.fit_transform(df_all), columns=df_all.columns)

In [112]:
# Encode the target variable - company name
label_encoder = LabelEncoder()
weighted_df['ISSUER_SLUG'] = label_encoder.fit_transform(weighted_df['ISSUER_SLUG'])

In [113]:
tfidf_vectorizer_legalentityname = TfidfVectorizer()
X_legalentityname = tfidf_vectorizer_legalentityname.fit_transform(weighted_df['LEGALENTITYNAME'].fillna(''))

In [114]:
tfidf_vectorizer_other_text = TfidfVectorizer()
X_other_text = tfidf_vectorizer_other_text.fit_transform(weighted_df['COMBINED_TEXT'])

In [115]:
X_combined = hstack([
    X_legalentityname,  
    X_other_text,       
     weighted_df[['DOMICILECOUNTRYCODE_WEIGHT', 'DOMICILESTATECODE_WEIGHT', 'SHARE_TYPE_WEIGHT', 'FUNDING_DATE_WEIGHT',
        'FORGE_PRICE_SOURCE_WEIGHT', 'FORGE_PRICE_ISSUER_TIER_WEIGHT', 'FORGE_PRICE_WEIGHT', 'FORGE_IMPLIED_VALUATION_WEIGHT']]
])



# Target 
y = weighted_df['ISSUER_SLUG']


In [134]:
rf_model = RandomForestClassifier()
rf_model.fit(X_combined, y)

RandomForestClassifier()

In [142]:
def predict_issuer_slug(input_text):
    input_legalentityname_tfidf = tfidf_vectorizer_legalentityname.transform([input_text])
    
    # For prediction, we don't have access to the other text fields, so use zeroed-out features for them
    default_weights = np.array([0] * 8).reshape(1, -1)
    default_other_text = csr_matrix(np.zeros((1, X_other_text.shape[1])))  # Empty additional text fields

    input_combined = hstack([input_legalentityname_tfidf, default_other_text, csr_matrix(default_weights)])
    prediction_encoded = rf_model.predict(input_combined)
    predicted_issuer_slug = label_encoder.inverse_transform(prediction_encoded)
    return predicted_issuer_slug[0]



In [144]:
# input_text = "actifio"
# input_text = "adaptive"
input_text = "airspace"
predicted_slug = predict_issuer_slug(input_text)
print(f"Predicted issuer slug: {predicted_slug}")

input_text = "FloQast Inc."
predicted_slug = predict_issuer_slug(input_text)
print(f"Predicted issuer slug: {predicted_slug}")

Predicted issuer slug: airspace-technologies-1
Predicted issuer slug: 6waves


In [ ]:
# Get top 3 predictions

def predict_top_n_issuer_slugs(input_text, n=3):
    input_tfidf = tfidf_vectorizer.transform([input_text])
    
    # Adjust default_weights to match the 8 numerical features used during training
    default_weights = np.array([0] * 8).reshape(1, -1)

    default_weights_sparse = csr_matrix(default_weights)
    input_combined = hstack([input_tfidf, default_weights_sparse])
    probs = rf_model.predict_proba(input_combined)
    top_n_idx = np.argsort(probs[0])[-n:][::-1]
    
    # Decode predictions back to original issuer slugs
    top_n_issuer_slugs = label_encoder.inverse_transform(top_n_idx)
    
    return top_n_issuer_slugs


input_text = "aera"
top_n_slugs = predict_top_n_issuer_slugs(input_text, n=3)
print(f"Top 3 predicted issuer slugs: {top_n_slugs}")


In [87]:
# Get feature importance
importances = rf_model.feature_importances_

feature_names = tfidf_vectorizer.get_feature_names_out()  # for TF-IDF features
important_features = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)

# top 10 important features
for feature, importance in important_features[:10]:
    print(f"Feature: {feature}, Importance: {importance}")


Feature: nan, Importance: 0.0033186269123794475
Feature: com, Importance: 0.003144815201515348
Feature: inc, Importance: 0.00312327581156618
Feature: www, Importance: 0.002956504172054034
Feature: crunchbase, Importance: 0.00293418478870793
Feature: and, Importance: 0.0029307986306222476
Feature: is, Importance: 0.002860210993884649
Feature: organization, Importance: 0.002815125334729644
Feature: https, Importance: 0.002800393310803576
Feature: of, Importance: 0.0026048730706348354


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.3, random_state=42)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)

print("Classification Report:")
print(classification_report(y_test, y_pred))
